In [1]:
%matplotlib inline
import csv
import numpy as np
import pandas as pd
from scipy.signal import *
import matplotlib.pyplot as plt
from scipy.ndimage import interpolation
from scipy.stats import entropy

# k Nearest Neighbors Classifier
from sklearn.neighbors import KNeighborsClassifier

import time

largeur=15
hauteur=6

In [ ]:
step_type_array = ["brush_back","brush_forward","dig","heel","brush_back"]
size_array = [9,9,9,9,5]
ext = ".txt"
dancer = "Daniel"

step_thresh = 3000

for i in range(len(size_array)):
    step_type = step_type_array[i]
    size = size_array[i]
    
    path_prep =  "etapdance2" + "/" + dancer + "/" + step_type + "/" + step_type + "_raw" + "/" + step_type + "_raw_"

    for n in range(size+1):
        step_type_nb = str(n)
        ext = ".txt"
        data_input = path_prep + step_type_nb + ext

        data_raw = pd.read_csv(data_input,names=['Gyro0X (deg)','Gyro0Y (deg)','Gyro0Z (deg)','Accel0X (g)','Accel0Y (g)','Accel0Z (g)','Gyro1X (deg)','Gyro1Y (deg)','Gyro1Z (deg)','Accel1X (g)','Accel1Y (g)','Accel1Z (g)'], dtype='a') 

        # MPU0 6050 (talon)
        g0x = np.array(data_raw['Gyro0X (deg)'])[1:].astype(float)
        g0y = np.array(data_raw['Gyro0Y (deg)'])[1:].astype(float)
        g0z = np.array(data_raw['Gyro0Z (deg)'])[1:].astype(float)

        a0x = np.array(data_raw['Accel0X (g)'])[1:].astype(float)
        a0y = np.array(data_raw['Accel0Y (g)'])[1:].astype(float)
        a0z = np.array(data_raw['Accel0Z (g)'])[1:].astype(float)

        # MPU1 6050 (pointe)
        g1x = np.array(data_raw['Gyro1X (deg)'])[1:].astype(float)
        g1y = np.array(data_raw['Gyro1Y (deg)'])[1:].astype(float)
        g1z = np.array(data_raw['Gyro1Z (deg)'])[1:].astype(float)

        a1x = np.array(data_raw['Accel1X (g)'])[1:].astype(float)
        a1y = np.array(data_raw['Accel1Y (g)'])[1:].astype(float)
        a1z = np.array(data_raw['Accel1Z (g)'])[1:].astype(float)

#Signal en sortie du capteur int16 soit dans l'intervalle [-32'767;32'767]
#Pleine échelle accéléromètre +- 16g LSB/g = 2'048.0
#Pleine échelle gyroscope +- 2000deg/s LSB/deg/s = 16.4

#On veut passer à un intervalle uint8 de [0;255]

#dans un premier temps on peut ajouter 32767 pour passer en uint16 (donc lintervalle [0;65'535])


plt.figure(figsize=(largeur,hauteur))
plt.plot(a0z,label='Accel0Z')
plt.plot(a1z,label='Accel1Z')
plt.plot(g0z,label='Gyro0Z')
plt.plot(g1z,label='Gyro1Z')
plt.ylabel('amplitude (-)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
plt.show()

plt.figure(figsize=(largeur,hauteur))
plt.plot(abs(a0z),label='Accel0Z absolu')
plt.plot(abs(a1z),label='Accel1Z absolu')
plt.plot(g0z,label='Gyro0Y')
plt.plot(g1z,label='Gyro1Y')
plt.plot([0,len(g0y)],[step_thresh,step_thresh], label = 'seuil pas')
plt.ylabel('amplitude (-)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
plt.show()


In [ ]:
g0x_filt = []
g0y_filt = []
g0z_filt = []
g1x_filt = []
g1y_filt = []
g1z_filt = []

a0x_filt = []
a0y_filt = []
a0z_filt = []
a1x_filt = []
a1y_filt = []
a1z_filt = []


val_nb = 6

motion_thresh = 200

for i in range(len(g0y-1)):
    if abs(g0y[i]) > motion_thresh or abs(g1y[i]) > motion_thresh:
        g0x_filt.append(g0x[i])
        g0y_filt.append(g0y[i])
        g0z_filt.append(g0z[i])
        g1x_filt.append(g1x[i])
        g1y_filt.append(g1y[i])
        g1z_filt.append(g1z[i])
        
        a0x_filt.append(a0x[i])
        a0y_filt.append(a0y[i])
        a0z_filt.append(a0z[i])
        a1x_filt.append(a1x[i])
        a1y_filt.append(a1y[i])
        a1z_filt.append(a1z[i])

        if abs(a0z[i]) > step_thresh or abs(a1z[i]) > step_thresh:
            a0z_max = abs(a0z[i])
            a1z_max = abs(a1z[i])
            print(a0z_max)
            print(a1z_max)
            start = len(g0y_filt)-val_nb
            end = len(g0y_filt)
            break

#Gyroscopes
plt.figure(figsize=(largeur,hauteur))
plt.plot(g0x,label='Gyro0X')
plt.plot(g0y,label='Gyro0Y')
plt.plot(g0z,label='Gyro0Z')
plt.plot(g1x,label='Gyro1X')
plt.plot(g1y,label='Gyro1Y')
plt.plot(g1z,label='Gyro1Z')
plt.ylabel('amplitude (-)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
plt.show()
            
plt.figure(figsize=(largeur,hauteur))
plt.plot(g0x_filt,label='Gyro0X dynamique')
plt.plot(g0y_filt,label='Gyro0Y dynamique')
plt.plot(g0z_filt,label='Gyro0Z dynamique')
plt.plot(g1x_filt,label='Gyro1X dynamique')
plt.plot(g1y_filt,label='Gyro1Y dynamique')
plt.plot(g1z_filt,label='Gyro1Z dynamique')
plt.ylabel('amplitude (-)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
plt.show()

plt.figure(figsize=(largeur,hauteur))
plt.plot(g0x_filt[start:end],label='Gyro0X pris en compte')
plt.plot(g0y_filt[start:end],label='Gyro0Y pris en compte')
plt.plot(g0z_filt[start:end],label='Gyro0Z pris en compte')
plt.plot(g1x_filt[start:end],label='Gyro1X pris en compte')
plt.plot(g1y_filt[start:end],label='Gyro1Y pris en compte')
plt.plot(g1z_filt[start:end],label='Gyro1Z pris en compte')
plt.ylabel('amplitude (-)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
plt.show()

#Accéleromètres
plt.figure(figsize=(largeur,hauteur))
plt.plot(a0x,label='Accel0X')
plt.plot(a0y,label='Accel0Y')
plt.plot(a0z,label='Accel0Z')
plt.plot(a1x,label='Accel1X')
plt.plot(a1y,label='Accel1Y')
plt.plot(a1z,label='Accel1Z')
plt.ylabel('amplitude (-)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
plt.show()
            
plt.figure(figsize=(largeur,hauteur))
plt.plot(a0x_filt,label='Accel0X dynamique')
plt.plot(a0y_filt,label='Accel0Y dynamique')
plt.plot(a0z_filt,label='Accel0Z dynamique')
plt.plot(a1x_filt,label='Accel1X dynamique')
plt.plot(a1y_filt,label='Accel1Y dynamique')
plt.plot(a1z_filt,label='Accel1Z dynamique')
plt.ylabel('amplitude (-)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
plt.show()

plt.figure(figsize=(largeur,hauteur))
plt.plot(a0x_filt[start:end],label='Accel0X pris en compte')
plt.plot(a0y_filt[start:end],label='Accel0Y pris en compte')
plt.plot(a0z_filt[start:end],label='Accel0Z pris en compte')
plt.plot(a1x_filt[start:end],label='Accel1X pris en compte')
plt.plot(a1y_filt[start:end],label='Accel1Y pris en compte')
plt.plot(a1z_filt[start:end],label='Accel1Z pris en compte')
plt.ylabel('amplitude (-)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
plt.show()

In [ ]:
headers=['Gyro0X (deg)','Gyro0Y (deg)','Gyro0Z (deg)',
             'Accel0X (g)','Accel0Y (g)','Accel0Z (g)',
             'Gyro1X (deg)','Gyro1Y (deg)','Gyro1Z (deg)',
             'Accel1X (g)','Accel1Y (g)','Accel1Z (g)']

df = pd.DataFrame({'Gyro0X (deg)': g0x_filt[start:end],
                       'Gyro0Y (deg)': g0y_filt[start:end],
                       'Gyro0Z (deg)': g0z_filt[start:end],
                       'Accel0X (g)' : a0x_filt[start:end],
                       'Accel0Y (g)' : a0y_filt[start:end],
                       'Accel0Z (g)' : a0z_filt[start:end],
                       'Gyro1X (deg)': g1x_filt[start:end],
                       'Gyro1Y (deg)': g1y_filt[start:end],
                       'Gyro1Z (deg)': g1z_filt[start:end],
                       'Accel1X (g)' : a1x_filt[start:end],
                       'Accel1Y (g)' : a1y_filt[start:end],
                       'Accel1Z (g)' : a1z_filt[start:end]})

data_output = "etapdance2" + "/" + dancer + "/" + step_type + "/" + step_type + "_cut" + "/" + step_type + "_cut_" + step_type_nb + ext
df.to_csv(path_or_buf=data_output,index=False)

In [ ]:
g0x = 32767
g0x =  np.uint8((g0x + 32767)/256)
print(g0x)

In [ ]:
g0x_scaled = g0x/max(abs(g0y))
g0y_scaled = g0y/max(abs(g0y))
g0z_scaled = g0z/max(abs(g0y))

a0x_scaled = a0x/max(abs(a0z))
a0y_scaled = a0y/max(abs(a0z))
a0z_scaled = a0z/max(abs(a0z))

g1x_scaled = g1x/max(abs(g1y))
g1y_scaled = g1y/max(abs(g1y))
g1z_scaled = g1z/max(abs(g1y))

a1x_scaled = a1x/max(abs(a1z))
a1y_scaled = a1y/max(abs(a1z))
a1z_scaled = a1z/max(abs(a1z))

g0y_uint8 = (g0y_scaled*127+127).astype('uint8')


plt.figure(figsize=(largeur,hauteur))
plt.plot(g0x,label='Gyro0X')
plt.plot(g0y,label='Gyro0Y')
plt.plot(g0z,label='Gyro0Z')
plt.plot(g1x,label='Gyro1X')
plt.plot(g1y,label='Gyro1Y')
plt.plot(g1z,label='Gyro1Z')
plt.ylabel('amplitude (deg/sec)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
#plt.savefig(fname='couplage_AC_step.png')
plt.show()

plt.figure(figsize=(largeur,hauteur))
plt.plot(g0x_scaled,label='Gyro0X scaled')
plt.plot(g0y_scaled,label='Gyro0Y scaled')
plt.plot(g0z_scaled,label='Gyro0Z scaled')
plt.plot(g1x_scaled,label='Gyro1X scaled')
plt.plot(g1y_scaled,label='Gyro1Y scaled')
plt.plot(g1z_scaled,label='Gyro1Z scaled')
plt.ylabel('amplitude (deg/sec)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
plt.show()

plt.figure(figsize=(largeur,hauteur))
plt.plot(g0y_uint8,label='Gyro0Y uint8')
plt.ylabel('amplitude (deg/sec)')
plt.xlabel('échantillons (-)')
plt.autoscale(axis='x',tight=True);
plt.legend()
plt.show()

In [6]:
#code complet
step_type_array = ["brush_back","brush_forward","dig","heel","step"]
size_array = [29,29,29,29,29]

ext = ".txt"
dancer = "Ilyas"

val_nb = 6 # le nombre de valeurs par axes prises en compte dans la reconnaissance
motion_thresh = 50 # le seuil de gyroscope ou l'on considère que le danseur commence un mouvement
step_thresh = 3000

for i in range(len(size_array)):
    step_type = step_type_array[i]
    size = size_array[i]
    
    path_prep =  "etapdance2" + "/" + dancer + "/" + step_type + "/" + step_type + "_int" + "/" + step_type + "_int_"

    for n in range(size+1):
        step_type_nb = str(n)
        ext = ".txt"
        data_input = path_prep + step_type_nb + ext

        data_raw = pd.read_csv(data_input,names=['Gyro0X (deg)','Gyro0Y (deg)','Gyro0Z (deg)','Accel0X (g)','Accel0Y (g)','Accel0Z (g)','Gyro1X (deg)','Gyro1Y (deg)','Gyro1Z (deg)','Accel1X (g)','Accel1Y (g)','Accel1Z (g)'], dtype='a') 

        # MPU0 6050 (talon)
        g0x = np.array(data_raw['Gyro0X (deg)'])[1:].astype(float)
        g0y = np.array(data_raw['Gyro0Y (deg)'])[1:].astype(float)
        g0z = np.array(data_raw['Gyro0Z (deg)'])[1:].astype(float)

        a0x = np.array(data_raw['Accel0X (g)'])[1:].astype(float)
        a0y = np.array(data_raw['Accel0Y (g)'])[1:].astype(float)
        a0z = np.array(data_raw['Accel0Z (g)'])[1:].astype(float)

        # MPU1 6050 (pointe)
        g1x = np.array(data_raw['Gyro1X (deg)'])[1:].astype(float)
        g1y = np.array(data_raw['Gyro1Y (deg)'])[1:].astype(float)
        g1z = np.array(data_raw['Gyro1Z (deg)'])[1:].astype(float)

        a1x = np.array(data_raw['Accel1X (g)'])[1:].astype(float)
        a1y = np.array(data_raw['Accel1Y (g)'])[1:].astype(float)
        a1z = np.array(data_raw['Accel1Z (g)'])[1:].astype(float)
        
        g0x_filt = []
        g0y_filt = []
        g0z_filt = []
        g1x_filt = []
        g1y_filt = []
        g1z_filt = []

        a0x_filt = []
        a0y_filt = []
        a0z_filt = []
        a1x_filt = []
        a1y_filt = []
        a1z_filt = []

        for i in range(len(g0y-1)):
            if abs(g0y[i]) > motion_thresh or abs(g1y[i]) > motion_thresh:
                g0x_filt.append(g0x[i])
                g0y_filt.append(g0y[i])
                g0z_filt.append(g0z[i])
                g1x_filt.append(g1x[i])
                g1y_filt.append(g1y[i])
                g1z_filt.append(g1z[i])

                a0x_filt.append(a0x[i])
                a0y_filt.append(a0y[i])
                a0z_filt.append(a0z[i])
                a1x_filt.append(a1x[i])
                a1y_filt.append(a1y[i])
                a1z_filt.append(a1z[i])

                if abs(a0z[i]) > step_thresh or abs(a1z[i]) > step_thresh:
                    a0z_max = abs(a0z[i])
                    a1z_max = abs(a1z[i])
                    #print(a0z_max)
                    #print(a1z_max)
                    start = len(g0y_filt)-val_nb
                    end = len(g0y_filt)
                    break
                    
        headers=['Gyro0X (deg)','Gyro0Y (deg)','Gyro0Z (deg)',
                 'Accel0X (g)','Accel0Y (g)','Accel0Z (g)',
                 'Gyro1X (deg)','Gyro1Y (deg)','Gyro1Z (deg)',
                 'Accel1X (g)','Accel1Y (g)','Accel1Z (g)']

        df = pd.DataFrame({'Gyro0X (deg)': g0x_filt[start:end],
                           'Gyro0Y (deg)': g0y_filt[start:end],
                           'Gyro0Z (deg)': g0z_filt[start:end],
                           'Accel0X (g)' : a0x_filt[start:end],
                           'Accel0Y (g)' : a0y_filt[start:end],
                           'Accel0Z (g)' : a0z_filt[start:end],
                           'Gyro1X (deg)': g1x_filt[start:end],
                           'Gyro1Y (deg)': g1y_filt[start:end],
                           'Gyro1Z (deg)': g1z_filt[start:end],
                           'Accel1X (g)' : a1x_filt[start:end],
                           'Accel1Y (g)' : a1y_filt[start:end],
                           'Accel1Z (g)' : a1z_filt[start:end]})

        data_output = "etapdance2" + "/" + dancer + "/" + step_type + "/" + step_type + "_cut_int" + "/" + step_type + "_cut_int_" + step_type_nb + ext
        df.to_csv(path_or_buf=data_output,index=False)
        